In [137]:
import json
from pathlib import Path
import collections
import fiona

from src.graph.graph_generator import GraphGenerator
from src.graph.path_schema import AttractionNode, AttractionsPair, GraphPath, PairsList

# Wczytywanie grafów
- grafu reprezentującego ściezki rowerowe, które zostały połączone z atrakcjami
- grafu reprezentującego drogi, chodniki itp., które zostały połączone z atrakcjami

In [2]:
graph_generator = GraphGenerator()

roads_graph = graph_generator.read_graph("data/roads_w_attractions.pickle")
bike_paths_graph = graph_generator.read_graph("data/bikepaths_w_attractions.pickle")

# Wczytywanie plików z zapisanymi trasami pomiędzy atrakcjami

In [19]:
PATHS_DIR = "out"

paths_dir = Path(PATHS_DIR)

paths_files = list(paths_dir.glob("data*.json"))

print("\n".join(p.name for p in paths_files))

data18.json
data7.json
data11.json
data12.json
data20.json
data8.json
data14.json
data22.json
data15.json
data3.json
data28.json
data17.json
data9.json
data5.json
data4.json
data16.json
data24.json
data21.json
data13.json
data19.json
data1.json
data23.json
data6.json
data2.json
data0.json
data26.json
data25.json
data27.json
data10.json


In [21]:
files_contents = PairsList(pairs=[])

for file in paths_files:
    content = PairsList.parse_file(file)
    files_contents.pairs = [*files_contents.pairs, *content.pairs]

In [22]:
len(files_contents.pairs)

29029

In [47]:
for pair in files_contents.pairs[:20]:
    len1 = pair.other_path.length if pair.other_path else float("inf")
    len2 = pair.bike_path.length if pair.bike_path else float("inf")
    len3 = pair.safer_bike_path.length if pair.safer_bike_path else float("inf")
    start_title = pair.start.name[:20] if pair.start.name else pair.start.id
    end_title = pair.end.name[:20] if pair.end.name else pair.end.id
    print(f"Z: {start_title:>20} do: {end_title:>20}", end=" -  ")
    print(f"Roads: {len1:7.3f} km,\tBike Lanes: {len2:7.3f} km,\tSafer Bike Lanes:{len3:7.3f} km")

Z:       Mangaba czarna do:              O'Barek -  Roads:   3.449 km,	Bike Lanes:   3.546 km,	Safer Bike Lanes:  3.574 km
Z:               Likaon do:                  609 -  Roads:   3.463 km,	Bike Lanes:   3.777 km,	Safer Bike Lanes:    inf km
Z:         Rzeźba Konia do:              Gaviusz -  Roads:   2.874 km,	Bike Lanes:   3.306 km,	Safer Bike Lanes:    inf km
Z:               Żyrafa do:           100matolog -  Roads:   2.666 km,	Bike Lanes:   3.349 km,	Safer Bike Lanes:  3.349 km
Z:     Bartek Skrzyński do:                  615 -  Roads:   5.316 km,	Bike Lanes:   6.667 km,	Safer Bike Lanes:  7.542 km
Z:              Słupnik do:                  448 -  Roads:   7.460 km,	Bike Lanes:     inf km,	Safer Bike Lanes:    inf km
Z:         Elektroludki do:                  485 -  Roads:     inf km,	Bike Lanes:   6.586 km,	Safer Bike Lanes:  6.586 km
Z:                Fenek do:                  559 -  Roads:   3.598 km,	Bike Lanes:   3.731 km,	Safer Bike Lanes:  4.029 km
Z:            Kr

# Sprawdźmy jaki był stosunek długości dróg

In [59]:
def get_ratios(p1, p2, p3):
    len1 = p1.length if p1 else None
    len2 = p2.length if p2 else None
    len3 = p3.length if p3 else None
    ratio1 = 0.0
    ratio2 = 0.0
    if len1 is not None and len2 is not None:
        ratio1 = len2 / len1
    if len1 is not None and len3 is not None:
        ratio2 = len3 / len1
    return ratio1, ratio2

ratios = [(i, *get_ratios(pair.other_path, pair.bike_path, pair.safer_bike_path)) for i, pair in enumerate(files_contents.pairs)]
ratios_filtered = list(filter(lambda x: x[1] != 0 or x[2] != 0, ratios))

In [60]:
print(f"Ratios len: {len(ratios)}, filtered len: {len(ratios_filtered)}")

Ratios len: 29029, filtered len: 27213


In [61]:
ratios_filtered

[(0, 1.0280792564183763, 1.036294914472829),
 (1, 1.0907433704224565, 0.0),
 (2, 1.1503307118046953, 0.0),
 (3, 1.256218758315874, 1.256218758315874),
 (4, 1.254025310651879, 1.4187207582366461),
 (7, 1.0368676105773864, 1.1196860022543242),
 (8, 1.128712438846799, 1.1464271724514121),
 (9, 1.0290835655274329, 0.0),
 (10, 1.0230372834183237, 1.0613583305363705),
 (11, 0.9901976942829223, 0.990863363282908),
 (12, 1.0083979864931683, 1.0083979864931683),
 (13, 1.0582203441192586, 1.0582203441192586),
 (14, 1.259439865626084, 1.259439865626084),
 (15, 1.0402625105141299, 1.1731615445059713),
 (16, 1.0091789395996558, 1.111823381156159),
 (17, 1.0785469882070753, 1.1843679592911909),
 (18, 0.9721796839726117, 1.125461805084207),
 (19, 0.9717256287608489, 0.9748466481166905),
 (20, 1.0219652064712064, 1.049757811472526),
 (21, 1.1014944151672972, 1.324439403973302),
 (22, 1.1490621463397075, 0.0),
 (23, 1.1480137181945995, 0.0),
 (24, 1.0869331182175948, 0.0),
 (25, 1.0346894060679777, 1.3

# Algorytm znajdowania braków w infrastrukturze rowerowej
Spróbujmy jakoś uszeregować ścieżki znalezione pomiędzy atrakcjami ze względu na dostępność dla rowerów

In [63]:
max(ratios_filtered, key=lambda x: x[2])

(4104, 1.015611407209453, 4.036488944296039)

In [85]:
max_ratio1 = max(ratios_filtered, key=lambda x: x[1])[1]
max_ratio2 = max(ratios_filtered, key=lambda x: x[2])[2]
points = {}
points1 = {}

for pair_index, ratio1, ratio2 in ratios_filtered:
    pair =files_contents.pairs[pair_index]
    if pair.other_path is not None and pair.bike_path is not None:
        ratio = ratio1 if ratio1 != 0 else max_ratio1
        for (i, j) in zip(pair.other_path.edges, pair.other_path.edges[1:]):
            d_points = ratio - 1.0 if ratio >= 1.0 else 0.0
            points[(i, j)] = points.get((i, j), 0) + d_points

    if pair.bike_path is not None and pair.safer_bike_path is not None:
        ratio = ratio2 if ratio2 != 0 else max_ratio2
        for (i, j) in zip(pair.bike_path.edges, pair.bike_path.edges[1:]):
            d_points = ratio - 1.0 if ratio >= 1.0 else 0.0
            points1[(i, j)] = points.get((i, j), 0) + ratio - 1.0

In [86]:
max(points.items(), key=lambda x: x[1])

((1077, 1078), 251.3760232682964)

In [88]:
max(points1.items(), key=lambda x: x[1])

((2385, 2384), 170.53064691266096)

In [111]:
missing_bikelanes = {}
for k, v in points.items():
    if missing_bikelanes.get(v) is None:
        missing_bikelanes[v] = []
    missing_bikelanes[v].append(k)

missing_bikelanes = collections.OrderedDict(sorted(missing_bikelanes.items()))

In [117]:
worst = list(filter(lambda x: x[0] >= 100, missing_bikelanes.items()))

In [122]:
points_ids = [i[1] for i in worst]

In [127]:
points_ids[:4]

[[(26595, 26596)],
 [(124726, 124727), (124727, 124728), (124728, 124729), (124729, 196276)],
 [(7445, 7444),
  (7444, 7443),
  (7443, 7442),
  (7442, 7441),
  (7441, 7440),
  (7440, 31468),
  (31468, 7451)],
 [(7447, 7446), (7446, 7445)]]

In [128]:
paths_nodes = []

for path in points_ids:
    path_nodes = []
    for i, j in path:
        if len(path_nodes) == 0 or path_nodes[-1] != i:
            path_nodes.append(i)
        if path_nodes[-1] != j:
            path_nodes.append(j)
    paths_nodes.append(path_nodes)

In [135]:
paths_coords = [[roads_graph.nodes[node]["pos"] for node in path] for path in paths_nodes]

In [138]:
with fiona.open('data/lines.shp') as src:
    meta = src.meta

In [145]:
with fiona.open("out/propose_bike_paths.shp", "w", **meta) as dst:
    for i, path in enumerate(paths_coords):
        path = {
            "type": "Feature",
            "id": i + 1,
            "properties": collections.OrderedDict([('osm_id', f'{i + 1000000}'),
               ('name', None),
               ('highway', 'motorway'),
               ('waterway', None),
               ('aerialway', None),
               ('barrier', None),
               ('man_made', None),
               ('z_order', 9),
               ('other_tags',
                '')]),
            "geometry": {
                "type": "LineString",
                "coordinates": path
            }
        }
        dst.write(path)

In [119]:
next(it)

(1,
 {'type': 'Feature',
  'id': '1',
  'properties': OrderedDict([('osm_id', '2512258'),
               ('name', None),
               ('highway', 'motorway'),
               ('waterway', None),
               ('aerialway', None),
               ('barrier', None),
               ('man_made', None),
               ('z_order', 9),
               ('other_tags',
                '"bdouble"=>"yes","destination:lanes"=>"Görlitz;Dresden|Görlitz;Dresden|Görlitz;Dresden|Magdeburg;Halle;Leipzig","destination:ref:lanes"=>"A 4|A 4|A 4|A 14","destination:symbol:lanes"=>"motorway|motorway|motorway|motorway","hazmat"=>"designated","int_r')]),
  'geometry': {'type': 'LineString',
   'coordinates': [(13.3684161, 51.0543897),
    (13.3703854, 51.0552565),
    (13.3704403, 51.0552807)]}})

In [140]:
next(it)

(2,
 {'type': 'Feature',
  'id': '2',
  'properties': OrderedDict([('osm_id', '2512517'),
               ('name', None),
               ('highway', 'motorway'),
               ('waterway', None),
               ('aerialway', None),
               ('barrier', None),
               ('man_made', None),
               ('z_order', 9),
               ('other_tags',
                '"bdouble"=>"yes","destination:lanes"=>"Chemnitz;Leipzig;Dreieck Nossen|Chemnitz;Leipzig;Dreieck Nossen|Chemnitz;Leipzig;Dreieck Nossen|Wilsdruff;Meißen;Tharandt","hazmat"=>"designated","int_ref"=>"E 40","lanes"=>"4","lit"=>"no","maxspeed"=>"none","onewa')]),
  'geometry': {'type': 'LineString',
   'coordinates': [(13.5260406, 51.0606644), (13.5244848, 51.060631)]}})